# Visualize a scattering event using my WebGL code

First, clone the GitHub repo: https://github.com/ageller/Scatter_WebGL

In [1]:
import os
import json
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

from IPython.display import IFrame

import http.server
import socketserver

from threading import Thread

In [2]:
# define the directory where the code lives (this can be anywhere on your computer; just change the directory below)
directory = os.path.join("/home/ynad93/tsunami/cusp_data/testing_files")

## Create some new data to replace the default data packaged with repo

For this example, I will read in some data in a csv file that I created a while ago.  (Note that this data is downsampled, and in some parts of the time sequence the trajectories do not look smooth.)

The data will need to be reformatted into a dict and output as a json with the following structure.  Each primary key is a time, and for each time there are particles that have the word "Particle" in the key name.  Each particle has at least a key of "r" with [x,y,z] values (anything else is ignored).

Below is a visual example of how the data should be formatted:

```
{
"0.1": {
  "Particle1": {
    "r" : [-44.59797843122882, -1.602469437175624, -6.039267952136012e-14],
   },
  "Particle2": {
    "r" : [21.90496690915433, 1.108629663443225, -0.01596151404716814],
   },
  "Particle3": {
    "r" : [22.6930115220745, 0.4938397737323992, 0.01596151404722853],
   }
},
"0.2": { ...
```

In [3]:
# read in the data
times = np.load(os.path.join(directory, "t.npy"))/(2*np.pi)
posall = np.load(os.path.join(directory, "pos.npy"))
starIDs = np.arange(posall.shape[1])

In [4]:
interp = interp1d(times, posall, kind='linear', axis=0)

In [5]:
tlin = np.arange(times[0], times[-1], 1E-3)
#tlin = np.linspace(times[0], times[-1], 25000)
f = interp(tlin)

In [10]:
where = np.where(np.mean(np.linalg.norm(posall, axis=-1), axis=-1)<100)

In [24]:
tlin.shape

(644663,)

In [22]:
f[0],f[-1]

(array([[ -131.24833751,  -270.43762333,   180.91977472],
        [ -131.22395685,  -270.54757403,   181.04227162],
        [ -129.5000651 ,  -273.23707974,   182.21088413],
        [   -4.68000118,  -434.60442039,   222.97729437],
        [  686.21139949,  1623.96845219, -1050.93375304],
        [  693.1035266 ,  1617.01436461, -1053.86444136]]),
 array([[   -27.74281677,    199.70811859,    442.51651552],
        [   -27.70526815,    199.69350322,    442.51022752],
        [  -508.43886638,  -9896.02954009, -14833.16972165],
        [   -31.6830949 ,    151.4977602 ,    435.8127425 ],
        [   726.04168318,   3171.55145606,   3014.85231992],
        [  -118.60597541,    259.87446931,    468.04115826]]))

In [12]:
# reformat and output to the json file
#Nframes = times.size
i0 = 0
i1 = -1


newData = dict()
i = i0
posallfin = f
timesfin = tlin
for t in timesfin[i0:i1]:
    newData[t] = {}
    j = 0
    for iden in starIDs:
        newData[t]['Particle' + str(iden)] = {'r': [posallfin[i,j,0], posallfin[i,j,1], posallfin[i,j,2]]}
        j+=1
    i+=1

# serialize json
json_object = json.dumps(newData, indent = 4)
 
# write to file (ScatterParts.json is the file name, and it lives in the code's data directory)
# Note: this will replace the default data set
with open(os.path.join(os.getcwd(),'data','ScatterParts.json'), "w") as outfile:
    outfile.write(json_object)

In [14]:
f.shape

(644663, 6, 3)

In [18]:
# define the port that you want (8000 is standard)
port = 8002

In [19]:
# a simple class to start the http server in a thread (so that you still have access to the jupyter notebook)
class serverThread(Thread): 
    def run(self):
        handler = http.server.SimpleHTTPRequestHandler
        #os.chdir(directory)
        with socketserver.TCPServer(("", port), handler) as httpd:
            print("serving at port", port)
            httpd.serve_forever()

serverThread().start()

serving at port 8002


In [20]:
# create an iFrame to view the visualization in this notebook
IFrame("http://localhost:8002", width = 1200, height = 800)

127.0.0.1 - - [07/Feb/2023 09:33:30] "GET / HTTP/1.1" 304 -
127.0.0.1 - - [07/Feb/2023 09:33:30] "GET /css/index.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Feb/2023 09:33:30] "GET /resources/three.min.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Feb/2023 09:33:30] "GET /resources/Detector.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Feb/2023 09:33:30] "GET /resources/KeyboardState.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Feb/2023 09:33:30] "GET /resources/TrackballControls.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2023 09:33:30] "GET /resources/THREEx.FullScreen.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Feb/2023 09:33:30] "GET /resources/THREEx.WindowResize.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Feb/2023 09:33:31] "GET /resources/THREE.MeshLine.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Feb/2023 09:33:31] "GET /resources/StereoEffect.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Feb/2023 09:33:31] "GET /resources/EffectComposer.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Feb/2023 09:33:31] "GET /resources/CopyShader.js HTTP/1.1" 304 -
127.